<a href="https://colab.research.google.com/github/brock-ricker/food_sales_predictions/blob/main/Project_1_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
#Importing Data

import pandas as pd
filepath = "https://raw.githubusercontent.com/brock-ricker/food_sales_predictions/main/sales_predictions.csv"
df = pd.read_csv(filepath)


In [57]:
#Initial Data Exploration

#Question 1
print(f"{df.shape[0]} rows and {df.shape[1]} columns")

#Question 2
print(df.info())

8523 rows and 12 columns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB
None


In [58]:
from pandas._libs.hashtable import value_count
from pandas.core.algorithms import value_counts
#Dropping Duplicates

#Question 3 - finding # of duplicates
dups = df.duplicated()
print(value_counts(dups))
#This is coming back as false (non-dupicates) = 8523, which is the same number of rows as original table, meaning there are no duplicates
#another way to check is to try dropping duplicates and then check the size of the resulting table
df = df.drop_duplicates()
df.info()
#same size as original, no duplicates

False    8523
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.6+ KB


In [60]:
#Question 4/5 Identifying missing values
#To start with, I can see from the df info that Item weight and Outlet size both have null entries (missing values)

#Start with item weight:
#To fill these missing values, I want to see if I can get the pertinant information from other rows.
weights = df.groupby("Item_Identifier")["Item_Weight"].unique()
#skimming this series, looks like most items have at least 1 entry for weight, lets use those entries to fill in the blanks. Ill take the mean of each weight entry(per unique item), in case there are >1 non-blank
#This will, in effect, replace the blank item wieghts with the average weight of the same item.
weights = df.groupby("Item_Identifier")["Item_Weight"].mean().round(2)
#creating dictionairy out of these values
weight_replace = weights.to_dict()
#use this dictionairy to replace the null values
df["Item_Weight"] = df["Item_Weight"].fillna(df["Item_Identifier"].map(weight_replace))
print(df.info())
#now we only have 4 missing values. Since we are down to only 4 missing values out of 8500+, I would rather lose that data than risk is screwing up my analysis. I will most likely drop them, but lets deal with the Outlet size first, before making that decision.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8519 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.6+ KB
None


In [67]:
#Question 5 Identifying missing values cont...

#outlet size. I am going to try to use the same strategy as above for outlet size
size = df.groupby(["Outlet_Identifier","Outlet_Location_Type","Outlet_Type"])["Outlet_Size"].unique()
print(size)
#Thats not going to work, because the size of some outlets is unknown. And the size for a given outlet_location_type, or outlet_type is not consistent.
#If I was given this dataset in a professional setting, I would just pick up the phone and aska about these outlets, but since I don't know how this data will be used yet, I am going to just mark these missing values and "unknown"
df["Outlet_Size"] = df["Outlet_Size"].fillna("unknown")

#and lets drop the missing values now.
df = df.dropna(how = "any")

#Question 6 confirming no more missing data
df.info()
#no more missing data

Outlet_Identifier  Outlet_Location_Type  Outlet_Type      
OUT010             Tier 3                Grocery Store        [unknown]
OUT013             Tier 3                Supermarket Type1       [High]
OUT017             Tier 2                Supermarket Type1    [unknown]
OUT018             Tier 3                Supermarket Type2     [Medium]
OUT019             Tier 1                Grocery Store          [Small]
OUT027             Tier 3                Supermarket Type3     [Medium]
OUT035             Tier 2                Supermarket Type1      [Small]
OUT045             Tier 2                Supermarket Type1    [unknown]
OUT046             Tier 1                Supermarket Type1      [Small]
OUT049             Tier 1                Supermarket Type1     [Medium]
Name: Outlet_Size, dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8519 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------              

In [68]:
#Question 7 addressing  inconsistent data

#Lets look at unique values for each column and see if there are any oddities. I will skip the item identifier, because I would not be able to spot inconsistincies in those

#item _weight - i dont know the wieghts of these products, but I can look for anything funky, like really high values, or really low values
print(df["Item_Weight"].min(),df["Item_Weight"].max()) 
#seems reasonable... 21.35 is high for a food item, but not insane

4.555 21.35


In [73]:
#fat content
print(df["Item_Fat_Content"].unique())
#lets fix these, first lets create the replacement dict
fat_cont = {"low fat":"Low Fat",
            "LF":"Low Fat",
            "reg":"Regular"
            }

df["Item_Fat_Content"] = df["Item_Fat_Content"].replace(to_replace = fat_cont)

#and check it
print(df["Item_Fat_Content"].unique())
#looks good

['Low Fat' 'Regular' 'low fat' 'LF' 'reg']
['Low Fat' 'Regular']


In [74]:
#Item Visibility, again, don't know too much about this one, but I know its a %, so nothing can be >100 or<0
print(df["Item_Visibility"].min(),df["Item_Visibility"].max()) 
#checks out

0.0 0.328390948


In [75]:
#item type
print(df["Item_Type"].unique())
#looks good

['Dairy' 'Soft Drinks' 'Meat' 'Fruits and Vegetables' 'Household'
 'Baking Goods' 'Snack Foods' 'Frozen Foods' 'Breakfast'
 'Health and Hygiene' 'Hard Drinks' 'Canned' 'Breads' 'Starchy Foods'
 'Others' 'Seafood']


In [76]:
#item MRP
print(df["Item_MRP"].min(),df["Item_MRP"].max())
#prices are reasonable

31.29 266.8884


In [77]:
#year est.
print(df["Outlet_Establishment_Year"].min(),df["Outlet_Establishment_Year"].max())
#reasonable

1985 2009


In [78]:
# Outlet_Size, Outlet_Location_Type, Outlet_Type - I already reviewed dueing question 4/5/6 but lets check them again just to bsure real quick
print(df["Outlet_Size"].unique())
print(df["Outlet_Location_Type"].unique())
print(df["Outlet_Type"].unique())
#looking good

['Medium' 'unknown' 'High' 'Small']
['Tier 1' 'Tier 3' 'Tier 2']
['Supermarket Type1' 'Supermarket Type2' 'Grocery Store'
 'Supermarket Type3']


In [79]:
#outlet sales
print(df["Item_Outlet_Sales"].min(),df["Item_Outlet_Sales"].max())
#these numbers don't make sense to me, but im not prepared to do anything with them until I learn more about them.

33.29 13086.9648
